# Task 4.1 Demonstrate execution Modes

## Objective 1: Execute on hardware

### Overview

Executing quantum circuits on real hardware involves several key steps:
1. **Circuit Preparation**: Create and optimize quantum circuits initialized locally or on cloud
2. **Backend Selection**: Choose appropriate quantum hardware
3. **Job Submission**: Send circuits to the quantum processor in batch or session
4. **Result Retrieval**: Obtain and analyze measurement outcomes

## Objective 2: Execution modes

### Three Execution Modes

IBM Quantum provides three distinct execution modes for running quantum programs:

1. **Job Mode** (`mode=backend`)
   - Single request of Sampler or Estimator without a context manager
   - Simplest mode for one-off executions
   - Specify `mode=backend` when initializing the primitive 

2. **Batch Mode** (`mode=batch`)
   - Multi-job workload made up of independently executable jobs, no condoitional relations to each other
   - Jobs are scheduled together for efficiency
   - Not gauranteed to run in order of submission, may run in parallel to other users jobs , QPU calibriation jobs may also run between batched jobs
   - Queuing time doesn't decrease for the first job submitted in a job
   - Specify `mode=batch` when initializing the primitive

3. **Session Mode** (`mode=session`)
   - Dedicated window to run a multi-job workload
   - execlusive access to the QPU and other jobs can run including calibriation jobs
   - Particularly useful for iterative tasks that require frequent communication between classical and quantum resources
   - Queuing time doesn't decrease for the first job submitted in a job
   - Specify `mode=session` when initializing the primitive

## Objective 3: Choose execution mode

### When to Use Each Mode

| Mode | Best For | Key Features |
|------|----------|-------------|
| **Batch** | Large collections of independent circuits | • Entire batch scheduled together<br>• Classical computations run in parallel<br>• Minimal delay between jobs<br>• Flexible - cancel remaining jobs if needed<br>• Less expensive than sessions |
| **Session** | Variational algorithms, iterative workloads | • All batch mode functionality<br>• Dedicated access to QPU<br>• Useful when inputs arrive gradually<br>• Better for workloads requiring low latency |
| **Job** | Small experiments, quick tests | • Simplest interface<br>• Might run sooner than batch<br>• Good for learning and testing |

### Best Practices

- Use batch mode by default, especially when all inputs are ready, it is more cost-effective than sessions.

- Batch mode is ideal for submitting multiple primitive jobs at once to reduce total runtime.

- Session mode is best for iterative workflows or when dedicated QPU access is required, but it is generally more expensive.

- Job mode should be used only for a single primitive request.

- Open Plan users cannot use sessions.

### Recommendations:

- Each job has fixed overhead. If a job uses less than 1 minute of QPU time, combine multiple tasks into a single job.

- If jobs take more than 1 minute or cannot be combined, submit multiple jobs in batch or session mode to exploit parallel classical processing (up to 5 jobs), even though the QPU runs one job at a time.

- Tune workload size and parallelism specially in sessions to minimize wall-clock time and cost; splitting large jobs can be more efficient.

## Objective 4 : Run Jobs in Batch

In [ ]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)
 
service = QiskitRuntimeService()

In [ ]:
backend = service.least_busy(operational=True, simulator=False)
batch = Batch(backend=backend)
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
# Close the batch because no context manager was used.
batch.close()

In [ ]:
from qiskit_ibm_runtime import (
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)
 
backend = service.least_busy(operational=True, simulator=False)
with Batch(backend=backend):
    estimator = Estimator()
    sampler = Sampler()

Batch Length is 10 minutes for open plan , 8 hours for paid plans

In [ ]:
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    job1 = estimator.run([estimator_pub])
    job2 = sampler.run([sampler_pub])
 
# The batch is no longer accepting jobs but the submitted job will run to completion.
result = job1.result()
result2 = job2.result()

In [ ]:
batch = Batch(backend=backend)
 
# If using qiskit-ibm-runtime earlier than 0.24.0, change `mode=` to `batch=`
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
job1 = estimator.run([estimator_pub])
job2 = sampler.run([sampler_pub])
print(f"Result1: {job1.result()}")
print(f"Result2: {job2.result()}")
 
# Manually close the batch. Running and queued jobs will run to completion.
batch.close()

In [ ]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    batch,
    SamplerV2 as Sampler,
)
 
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
 
with Batch(backend=backend) as batch:
    print(batch.details())

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler, Batch
from qiskit.circuit.random import random_circuit
 
max_circuits = 100
circuits = [random_circuit(5, 5) for _ in range(5 * max_circuits)]
all_partitioned_circuits = []
for i in range(0, len(circuits), max_circuits):
    all_partitioned_circuits.append(circuits[i : i + max_circuits])
jobs = []
start_idx = 0
 
with Batch(backend=backend):
    sampler = Sampler()
    for partitioned_circuits in all_partitioned_circuits:
        job = sampler.run(partitioned_circuits)
        jobs.append(job)